# VASP bandstructure workflow using MatMethods

In this excercise we will use the [MatMethods](https://github.com/hackingmaterials/MatMethods.git) package to compute the bandstructre of silicon.

**MatMethods:**

- Successor to **MPWorks**: the software framework that powers the MaterialsProject

- Like MPWorks it is built on top of state-of-the-art open-source libraries: **pymatgen, custodian, and FireWorks**
    
- Unlike MPWorks, MatMethods provides a much **cleaner and simpler interface** for composing complex workflows.

- Features:
 
    - Provides “standard” workflows for a wide variety of desired materials properties - optimized structures, band structures, electronic transport properties, dielectric constants, and much more. Just provide a crystal structure (that’s it!) and let MatMethods set up a complete workflow that provides the property you are interested in. You can do this for a single material, 100 materials, or 100,000 materials.

    - One can easily change “standard workflows” - whether that is changing some of the default calculation parameters or recomposing the workflow (adding new calculations, removing steps, etc.) - using a very expressive syntax. One can compose very complex new workflows simply by chaining together pre-built calculation steps.

    - It can build large databases of output properties that you can query, analyze, and share in a systematic way.

    - It automatically keeps meticulous records of jobs, their directories, runtime parameters, etc.

    - Jobs can be run on a variety of computing systems, queue systems, and architectures.
    
    - Uses a standard interface for adding new types of calculations and workflows such that it is possible for users to contribute new features and grow the capabilities of the software over time.

# Setup

- fire up a terminal
- switch to python2 environment
    - source activate python2
- get the latest version of MatMethods and install
    - git clone https://github.com/hackingmaterials/MatMethods.git
    - cd MatMethods
    - python setup.py develop
- start mongoDB
    - mongod

# Set environment variables

The environment variable VASP_PSP_DIR points to the location of VASP potcar files.

In [ ]:
import os
import json

from pymongo import MongoClient

#set the path to vasp pseudopotential files
os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "Si_bandstructure_runs")

# Get Silicon Structure

In [ ]:
from pymatgen.util.testing import PymatgenTest

struct_si = PymatgenTest.get_structure("Si")
print struct_si

# Set vasp inputset: incar, poscar, potcar and kpoints

In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet

vis = MPRelaxSet(struct_si, force_gamma=True)

print vis.incar
print vis.poscar
print vis.potcar_functional, vis.potcar_symbols
print vis.kpoints

# Define Fireworks and Workflow

The workflow consists of  3 fireworks:
- structure optimization
- static calculation
- refined band structure calculation

Structure optimization firework

In [ ]:
from matmethods.vasp.fireworks.core import OptimizeFW

fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<")

The Structure optimization firework consists of the following firetasks

In [ ]:
for task in fw1.tasks:
    print task.fw_name

The Static firework

In [ ]:
from matmethods.vasp.fireworks.core import StaticFW

fw2 = StaticFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<", parents=[fw1])

for task in fw2.tasks:
    print task.fw_name

The Non-selfconsistent firework consists of the following firetasks

In [ ]:
from matmethods.vasp.fireworks.core import NonSCFFW

fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="line")

for task in fw3.tasks:
    print task.fw_name

Finally, the workflow

In [ ]:
from fireworks import Workflow

fws = [fw1, fw2, fw3]

wf = Workflow(fws)

# Emulate VASP

In this excersice due to time and resource constraints we will not be running vasp, instead emulate it using pre-computed results of the workflow. To that end we employ the 'use_fake_vasp' powerup to replace the vasp custodian task with the vasp emulator task.

In [ ]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

reference_dir = os.path.join(os.getcwd(), "Si_bandstructure_runs")
#reference_dir = os.path.abspath(os.path.join("..","..","matproj/MatMethods/matmethods/vasp/tests", "reference_files"))

si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si_structure_optimization"),
               "static": os.path.join(reference_dir, "Si_static"),
               "nscf uniform": os.path.join(reference_dir, "Si_nscf_uniform"),
               "nscf line": os.path.join(reference_dir, "Si_nscf_line")}

wf = use_fake_vasp(wf, si_ref_dirs)

# Connect to Launchpad and reset

Establish conncetion to the mongodb database that stores the workflow(the launchpad) and reset it.

In [ ]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

# Add Workflow

add the workflow to the launchpad

In [ ]:
lp.add_wf(wf)

Some utility functions

In [ ]:
def get_status():
    """
    print firework names and their states
    """
    for id in lp.get_fw_ids():
        fw = lp.get_fw_by_id(id)
        print "Name:{}, State:{}".format(fw.name, fw.state)

def set_launch_dir(launch_dir):
    os.mkdir(launch_dir)
    os.chdir(launch_dir)
    
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db[creds["collection"]]

print firework names and their states

In [ ]:
get_status()

# Set the Fireworker

The fireworker environemnt variable 'db_file' is set to the path to the mongodb connection setting to the databse that stores the outputs of each run.

In [ ]:
from fireworks import FWorker

db_file = os.path.join(os.getcwd(), "db.json")
my_fworker=FWorker(env={"db_file": db_file})

# Launch Structure optimization firework

create a launch directory, change to it and launch the first firework

Note: As shown in the previous session the firework launches and status checking can also be done using the commandline utility 'lpad'.

In [ ]:
from fireworks.core.rocket_launcher import launch_rocket

set_launch_dir("structure_optimization")
launch_rocket(lp, fworker=my_fworker)

In [ ]:
get_status()

# Launch static vasp calculation firework

In [ ]:
set_launch_dir("../static")

launch_rocket(lp, fworker=my_fworker)

In [ ]:
get_status()

# Launch vasp bandstructure calculation firework

In [ ]:
set_launch_dir("../bandstructure")

launch_rocket(lp, fworker=my_fworker)

# Retrieve data


In [ ]:
from pymongo import DESCENDING

d1 = get_task_collection(db_file).find_one({"task_label": "structure optimization"}, 
                                           sort=[("_id", DESCENDING)])

d2 = get_task_collection(db_file).find_one({"task_label": "static"}, 
                                           sort=[("_id", DESCENDING)])

d3 = get_task_collection(db_file).find_one({"task_label": "nscf uniform"}, 
                                           sort=[("_id", DESCENDING)])

d4 = get_task_collection(db_file).find_one({"task_label": "nscf line"}, 
                                           sort=[("_id", DESCENDING)])

Bandgap

In [ ]:
d2["output"]["bandgap"]